<a href="https://colab.research.google.com/github/JDA156/Hitss/blob/main/prueba_hitss_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba Hitss

In [3]:
# Configuración para ejecutar Selenium en Google Colab
## No es necesario ejecutar este código si lo hace en Jupyter Notebook u otra configuración local de Python.
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,798 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,557 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [4]:
!pip install chromedriver-autoinstaller
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import re
import chromedriver_autoinstaller  # configurar opciones de chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Asegúrese de que la GUI esté desactivada
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # Establezca la ruta a Chromedriver según su configuración
chromedriver_autoinstaller.install()  # establecer la URL de destino
driver = webdriver.Chrome(options=chrome_options)

In [6]:
# ==============================================================================
# SCRIPT DE AUTOMATIZACIÓN (V6 - USANDO RUTAS ORIGINALES DEL USUARIO)
# ==============================================================================
# Este script utiliza las rutas XPath exactas proporcionadas en la solicitud,
# incluyendo la lógica de 3 reintentos por si un clic es interceptado.
# ==============================================================================

import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

# --- Configuración Inicial ---
if not os.path.exists("informe_capturas"):
    os.makedirs("informe_capturas")

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--log-level=3')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
wait = WebDriverWait(driver, 20) # Aumentamos la espera a 20 segundos para dar tiempo a la página

# --- Función de Reintento de Clic ---
def safe_click_con_scroll(locator, description):
    """
    Intenta hacer scroll hasta el elemento y luego hace clic, reintentando hasta 3 veces.
    """
    for attempt in range(3):
        try:
            element = wait.until(EC.presence_of_element_located(locator))
            # Hacemos scroll para asegurar que el elemento esté visible
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", element)
            time.sleep(1) # Pequeña pausa para que se complete el scroll

            element_clickable = wait.until(EC.element_to_be_clickable(locator))
            element_clickable.click()

            print(f"Éxito al hacer clic en: '{description}'")
            return
        except Exception as e:
            print(f"Intento {attempt + 1} fallido para '{description}': {type(e)._name_}")
            if attempt < 2:
                print("Reintentando en 2 segundos...")
                time.sleep(2)
            else:
                print(f"FALLO DEFINITIVO: No se pudo hacer clic en '{description}' después de 3 intentos.")
                raise

# --- Inicio de la Prueba ---
try:
    driver.get('https://www.mercadolibre.com/')

    # Aceptar cookies
    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar cookies')]"))).click()
        print("Cookies aceptadas.")
        time.sleep(1)
    except TimeoutException:
        print("No se encontró el banner de cookies.")

    # =================================================
    # USANDO TUS RUTAS XPATH EXACTAS
    # =================================================

    # 1. Seleccionar país
    safe_click_con_scroll((By.XPATH, '//*[@id="MX"]'), "País México")
    time.sleep(2)
    driver.save_screenshot('informe_capturas/01_pais_seleccionado.png')

    # 2. Buscar "playstation 5"
    search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cb1-edit"]')))
    search_bar.send_keys('playstation 5')
    safe_click_con_scroll((By.XPATH, '/html/body/header/div/div[2]/form/button'), "Botón de búsqueda")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/02_resultados_busqueda.png')

    # 3. Aplicar filtro "Nuevo"
    print("Aplicando filtro 'Nuevo'...")
    safe_click_con_scroll((By.XPATH, '//*[@id="root-app"]/div/div[2]/aside/section[2]/ul/li[1]/a'), "Filtro Nuevo")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/03_filtro_nuevos.png')

    # 4. Aplicar filtro de ubicación "CDMX"
    print("Aplicando filtro de ubicación 'CDMX'...")
    safe_click_con_scroll((By.XPATH, '//*[@id="root-app"]/div/div[2]/aside/section[3]/ul/li[1]/a'), "Filtro Ubicación CDMX")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/04_filtro_ubicacion.png')

    # 5. Ordenar por precio
    print("Ordenando por precio...")
    # ADVERTENCIA: Esta ruta 'display-values' es dinámica y es la que seguramente fallará.
    safe_click_con_scroll((By.XPATH, '//*[@id=":R19l5e:"]/div/div/div'), "Desplegable para ordenar")
    time.sleep(2)
    # ADVERTENCIA: Esta ruta 'menu-list-option' también es dinámica.
    safe_click_con_scroll((By.XPATH, '//*[@id=":R19l5e:-menu-list-option-price_desc"]'), "Opción 'Mayor precio'")
    time.sleep(3)
    driver.save_screenshot('informe_capturas/05_ordenado_mayor_precio.png')

    # 6. Extraer los datos
    print("Extrayendo información de los 5 primeros productos...")
    items = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]")
    print("\n" + "="*60)
    print("       INFORME FINAL: 5 PRODUCTOS PLAYSTATION 5 MÁS CAROS")
    print("="*60)
    for i, item in enumerate(items[:5]):
        name = item.find_element(By.XPATH, ".//h2[contains(@class, 'ui-search-item__title')]").text
        price = item.find_element(By.XPATH, ".//span[contains(@class, 'andes-money-amount__fraction')]").text
        print(f"Producto {i+1}:\n  Nombre: {name}\n  Precio: ${price}\n")
    print("="*60)

except Exception as e:
    print("\n-------------------------------------------------------------")
    print("               EL SCRIPT HA FALLADO")
    print("-------------------------------------------------------------")
    print(f"\nEl error fue de tipo: {type(e)._name_}")
    error_filename = 'informe_capturas/error_screenshot.png'
    driver.save_screenshot(error_filename)
    print(f"Se guardó una captura del error en '{error_filename}'.")

finally:
    if 'driver' in locals() and driver:
        driver.quit()
    print("\nPrueba finalizada.")

No se encontró el banner de cookies.
Éxito al hacer clic en: 'País México'
Éxito al hacer clic en: 'Botón de búsqueda'
Aplicando filtro 'Nuevo'...

-------------------------------------------------------------
               EL SCRIPT HA FALLADO
-------------------------------------------------------------

Prueba finalizada.


AttributeError: type object 'AttributeError' has no attribute '_name_'